# Apply transfer learning

In [2]:
import wandb
from wandb.keras import WandbCallback
run = wandb.init(project='transfer_learning', entity='gijstimmerij', reinit=True)

wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "c:\Users\gijst\anaconda3\envs\tf-gpu\lib\site-packages\wandb\integration\keras\keras.py", line 77, in patch_tf_keras
    from keras.engine import training
  File "c:\Users\gijst\anaconda3\envs\tf-gpu\lib\site-packages\wandb\util.py", line 1344, in load_module
    mod = importlib.import_module(fullname)
  File "c:\Users\gijst\anaconda3\envs\tf-gpu\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
ModuleNotFoundError: No module named 'keras'
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gijstimmerij. Use `wandb login --relogin` to force relogin


In [3]:
import numpy as np # linear algebra
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import tensorflow.keras.backend as K


In [1]:
from importlib.machinery import SourceFileLoader
somemodule = SourceFileLoader('tcn', r'C:\Users\gijst\Documents\Uni\scriptie\RUL-prediction\keras-tcn-master\tcn\tcn.py').load_module()

### Import datasets

In [6]:
dataset = 'df3'
swp_X = np.load(f'Data/{dataset}/swpX.npy')
swp_y = np.load(f'Data/{dataset}/swpy.npy')
swp_X_test = np.load(f'Data/{dataset}/swpX_test.npy')
swp_y_test = np.load(f'Data/{dataset}/swpy_test.npy')
swp_X_reduced = np.load(f'Data/{dataset}/swpX_reducedn10.npy')
swp_y_reduced = np.load(f'Data/{dataset}/swpy_reducedn10.npy')

In [7]:
swp_X.shape

(17379, 16, 30)

### Import pretrained model

In [4]:
from tcn import TCN

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))


model_path = r"C:\Users\gijst\Documents\Uni\scriptie\Transfer Learning\models\swp_30_best\df1_ft64_ks3_do06.h5"
estimator = load_model(model_path, custom_objects={
                       'root_mean_squared_error': root_mean_squared_error, 'TCN': TCN})

In [7]:
estimator.summary()

Model: "model_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        [(None, 16, 30)]          0         
_________________________________________________________________
tcn_76 (TCN)                 (None, 16, 64)            171008    
_________________________________________________________________
tcn_77 (TCN)                 (None, 32)                44608     
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 33        
Total params: 215,649
Trainable params: 215,649
Non-trainable params: 0
_________________________________________________________________


### Set layers to be trained

In [22]:
group = 'df4_reducedn10_inilr.0005_redlr_alltrain'
swpX = swp_X_reduced
swpy = swp_y_reduced.astype('float')
adam = keras.optimizers.Adam(learning_rate=0.0005)
# adam = keras.optimizers.Adam()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, verbose=1)
for i in range(3):
    model = keras.Sequential()
    for layer in estimator.layers[:-1]:
        model.add(layer)
    model.add(Dense(1, activation='linear'))
    for layer in model.layers[:-1]:
        layer.trainable = True
    model.compile(loss=root_mean_squared_error, optimizer=adam,
                  metrics=['mae', root_mean_squared_error])
    run = wandb.init(project='transfer_learning1', entity='gijstimmerij',
                     reinit=True, group=group)
    history = model.fit(swpX, swpy, epochs=100, batch_size=200,
                        validation_split=.05, callbacks=[WandbCallback(), reduce_lr])
    run.name = f'{group}_{i+1}'
    run.finish()


Epoch 1/100
13/13 [==============================] - 6s 109ms/step - loss: 98.9602 - mae: 78.2073 - root_mean_squared_error: 98.3987 - val_loss: 36.5299 - val_mae: 30.4188 - val_root_mean_squared_error: 36.5299
Epoch 2/100
13/13 [==============================] - 0s 31ms/step - loss: 88.1895 - mae: 69.3635 - root_mean_squared_error: 88.3083 - val_loss: 26.6340 - val_mae: 20.6107 - val_root_mean_squared_error: 26.6340
Epoch 3/100
13/13 [==============================] - 0s 32ms/step - loss: 85.7744 - mae: 68.1045 - root_mean_squared_error: 86.0743 - val_loss: 30.4559 - val_mae: 24.6217 - val_root_mean_squared_error: 30.4559
Epoch 4/100
13/13 [==============================] - 0s 32ms/step - loss: 82.4651 - mae: 66.0927 - root_mean_squared_error: 81.9827 - val_loss: 32.4321 - val_mae: 26.5179 - val_root_mean_squared_error: 32.4321
Epoch 5/100
13/13 [==============================] - 0s 31ms/step - loss: 81.2409 - mae: 64.9561 - root_mean_squared_error: 82.1554 - val_loss: 30.5572 - val_m

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▁▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▂▁▁▂▂▁▁▁▂▂▁▁▂▂▁▂▁▁
mae,█▄▃▂▂▂▁▁▂▁▂▁▁▂▂▁▁▁▂▁▁▁▁▁▁▂▂▁▁▂▁▂▁▁▂▂▁▂▁▁
root_mean_squared_error,█▅▄▃▂▂▁▂▂▂▂▂▁▂▂▂▁▁▂▁▂▁▂▁▁▁▂▂▂▂▂▂▁▂▂▂▁▂▁▂
val_loss,▇▁▂▃▆▆██████████████████████████████████
val_mae,▆▁▂▃▆▆██████████████████████████████████
val_root_mean_squared_error,▇▁▂▃▆▆██████████████████████████████████
best_epoch,1
best_val_loss,26.63396
epoch,99
loss,74.9123


Epoch 1/100
13/13 [==============================] - 6s 109ms/step - loss: 274.9442 - mae: 248.9967 - root_mean_squared_error: 273.9099 - val_loss: 166.1506 - val_mae: 156.2057 - val_root_mean_squared_error: 166.1506
Epoch 2/100
13/13 [==============================] - 0s 30ms/step - loss: 275.0706 - mae: 249.3940 - root_mean_squared_error: 273.7471 - val_loss: 166.1504 - val_mae: 156.2055 - val_root_mean_squared_error: 166.1504
Epoch 3/100
13/13 [==============================] - 0s 30ms/step - loss: 275.9303 - mae: 250.0389 - root_mean_squared_error: 276.1931 - val_loss: 166.1502 - val_mae: 156.2053 - val_root_mean_squared_error: 166.1502
Epoch 4/100
13/13 [==============================] - 0s 31ms/step - loss: 274.7255 - mae: 248.7886 - root_mean_squared_error: 273.5388 - val_loss: 166.1501 - val_mae: 156.2051 - val_root_mean_squared_error: 166.1501
Epoch 5/100
13/13 [==============================] - 0s 31ms/step - loss: 273.9896 - mae: 248.4665 - root_mean_squared_error: 272.5484 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▄▆▅▁▄▅█▁▇▄▆▄▃▅▂█▄▃▃▁▃▃▅▁▂▂▄▇▅▂▅▇▄▇▄▅▄▃█▅
mae,▅▆▄▂▄▆█▁▇▅▆▄▃▅▂█▅▄▄▃▃▄▅▃▃▂▅▇▅▂▅█▅▇▅▄▆▄▇▄
root_mean_squared_error,▄▅▄▁▄▅▇▃▇▅▆▄▃▄▅▆▅▅▄▂▂▃▅▁▄▄▄▅▄▃▄▅▃▅▄▄▅▂█▃
val_loss,██▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_mae,██▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_root_mean_squared_error,██▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
best_epoch,99
best_val_loss,166.14507
epoch,99
loss,275.51566


Epoch 1/100
13/13 [==============================] - 6s 108ms/step - loss: 202.5136 - mae: 173.7394 - root_mean_squared_error: 203.7655 - val_loss: 106.8939 - val_mae: 95.6076 - val_root_mean_squared_error: 106.8939
Epoch 2/100
13/13 [==============================] - 0s 31ms/step - loss: 204.4080 - mae: 174.6368 - root_mean_squared_error: 204.3101 - val_loss: 106.8939 - val_mae: 95.6076 - val_root_mean_squared_error: 106.8939
Epoch 3/100
13/13 [==============================] - 0s 30ms/step - loss: 202.4839 - mae: 173.8932 - root_mean_squared_error: 201.9963 - val_loss: 106.8938 - val_mae: 95.6075 - val_root_mean_squared_error: 106.8938
Epoch 4/100
13/13 [==============================] - 0s 31ms/step - loss: 202.0702 - mae: 173.7493 - root_mean_squared_error: 202.9641 - val_loss: 106.8938 - val_mae: 95.6074 - val_root_mean_squared_error: 106.8938
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 203.9881 - mae: 174.9904 - root_mean_squared_error: 205.1221 - va

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▅▅▃▅▄▆▅▂▄▁▆▅▅▆▄▆▇▆▄▇▄█▄▆▅▅▅▅▇▄▅▄▆▄▆▃▆▅▇▆
mae,▅▅▄▅▅▅▆▂▄▁▅▆▅▆▅▇▇▇▄▇▅█▄▆▆▅▄▆▇▃▆▄▆▄▆▄▇▆█▆
root_mean_squared_error,▅▃▃▄▅▇▄▂▅▁▄▂▄▆▄▃▆▅▃▆▄█▃▆▃▄▄▄▆▃▃▂▅▄▆▁▄▅▆▆
val_loss,█▇▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▇▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▇▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,33
best_val_loss,106.89335
epoch,99
loss,203.94386


In [82]:
model_path = r"models\transfer_learning\df4_n10_redlr_alltrain.h5"
estimator = load_model(model_path,custom_objects={'root_mean_squared_error': root_mean_squared_error,'TCN': TCN})

In [83]:
estimator.evaluate(swp_X_test, swp_y_test.astype('float'), verbose=1)

129/129 [==============================] - 2s 8ms/step - loss: 58.6796 - mae: 55.7511 - root_mean_squared_error: 58.6823


[58.67958068847656, 55.75108337402344, 58.68227767944336]